To close out this checkpoint, you're going to do three assignments. For the first assignment, you'll write up a short answer to a question. For the second two assignments, you'll do your work in Jupyter notebooks.

Please submit links to all your work at the end of this assessment.

1. Interpretation and significance
Suppose that we would like to know how much families in the US are spending on recreation annually. We've estimated the following model:

 

𝑒𝑥𝑝𝑒𝑛𝑑𝑖𝑡𝑢𝑟𝑒=873+0.0012𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒+0.00002𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒2−223.57ℎ𝑎𝑣𝑒_𝑘𝑖𝑑𝑠expenditure=873+0.0012annual_income+0.00002annual_income2−223.57have_kids

expenditure is the annual spending on recreation in US dollars, annual_income is the annual income in US dollars, and have_kids is a dummy variable indicating the families with children. Interpret the estimated coefficients. What additional statistics should be given in order to make sure that your interpretations make sense statistically. Write up your answer.




The increase in annual income has a large positive impact on recreational spending, while having kids has a negative impact on spending. The p-value for each coefficient will determine how significant the individual feature has on predicting spending.

2. Weather model
In this exercise, you'll work with the historical temperature data from the previous checkpoint. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

First, load the dataset from the weatherinszeged table from Thinkful's database.
Build a linear regression model where your target variable is the difference between the apparenttemperature and the temperature. As explanatory variables, use humidity and windspeed. Now, estimate your model using OLS. Are the estimated coefficients statistically significant? Are the signs of the estimated coefficients in line with your previous expectations? Interpret the estimated coefficients. What are the relations between the target and the explanatory variables?
Next, include the interaction of humidity and windspeed to the model above and estimate the model using OLS. Are the coefficients statistically significant? Did the signs of the estimated coefficients for humidity and windspeed change? Interpret the estimated coefficients.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pylab
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [3]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

df = pd.read_sql_query('select* from weatherinszeged', con = engine)


engine.dispose()

In [4]:
df.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


In [30]:
y = df['temperature'] - df['apparenttemperature']
X = df[['humidity', 'windspeed']]
import statsmodels.api as sm

from statsmodels.tools.tools import add_constant
X = sm.add_constant(X)
results = sm.OLS(y, X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Tue, 26 May 2020   Prob (F-statistic):               0.00
Time:                        21:39:30   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.4381      0.021   -115.948      0.0

The feature coefficients are both statistically significant. These are in line with previous estimations in that the correlation coefficient is fairly low. It is hard to predict the target with such few parameters. It looks like higher humidity has a positive correlation with temperature. This makes sense in terms of dew point. More moisture can exist at higher temperatures.

In [31]:
X2 = pd.concat([X,(df['humidity']*df['windspeed']).rename('wind-hum')], axis = 1)


,const,humidity,windspeed,wind-hum
0,1.0,0.89,14.1197,12.566533
1,1.0,0.86,14.2646,12.267556
2,1.0,0.89,3.9284,3.496276
3,1.0,0.83,14.1036,11.705988
4,1.0,0.83,11.0446,9.167018
...,...,...,...,...
96448,1.0,0.43,10.9963,4.728409
96449,1.0,0.48,10.0947,4.845456
96450,1.0,0.56,8.9838,5.030928
96451,1.0,0.60,10.5294,6.317640


In [32]:
X = sm.add_constant(X)
results = sm.OLS(y, X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Tue, 26 May 2020   Prob (F-statistic):               0.00
Time:                        21:40:21   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0839      0.033     -2.511      0.0

3. House prices model
In this exercise, you'll interpret your house prices model. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

Load the houseprices data from Thinkful's database.
Run your house prices model again and interpret the results. Which features are statistically significant, and which are not?
Now, exclude the insignificant features from your model. Did anything change?
Interpret the statistically significant coefficients by quantifying their relations with the house prices. Which features have a more prominent effect on house prices?
Do the results sound reasonable to you? If not, try to explain the potential reasons.

In [37]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'



engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

df3 = pd.read_sql_query('select* from houseprices', con = engine)

engine.dispose()

df3.overallcond.value_counts()





np.abs(df3.corr().loc[:,"saleprice"]).sort_values(ascending=False)

# Set features and variables
X3 = pd.concat([df3[['overallcond', 'grlivarea', 'garagecars']], pd.get_dummies(df3['roofmatl'], prefix = 'roofmatl', drop_first = True)], axis = 1)
y3 = df3['saleprice']


# Find the OLS
import statsmodels.api as sm
X3 = sm.add_constant(X3)

results = sm.OLS(y3, X3).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.648
Model:                            OLS   Adj. R-squared:                  0.645
Method:                 Least Squares   F-statistic:                     266.4
Date:                Tue, 26 May 2020   Prob (F-statistic):          1.22e-319
Time:                        21:49:48   Log-Likelihood:                -17783.
No. Observations:                1460   AIC:                         3.559e+04
Df Residuals:                    1449   BIC:                         3.565e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -4.019e+05   4.98e+04     -8.069      0.000      -5e+05   -3.04e+05
overallcond       2444.1382   1135.139      2.153      0.031     217.447    4670.829
grlivarea           82.8402      2.771     29.901      0.000      77.406      88.275
garagecars        4.113e+04   1914.445     21.485      0.000    3.74e+04    4.49e+04
roofmatl_CompShg  3.709e+05   4.87e+04      7.622      0.000    2.75e+05    4.66e+05
roofmatl_Membran  4.311e+05    6.8e+04      6.339      0.000    2.98e+05    5.64e+05
roofmatl_Metal    4.055e+05   6.81e+04      5.951      0.000    2.72e+05    5.39e+05
roofmatl_Roll     2.853e+05   6.77e+04      4.214      0.000    1.53e+05    4.18e+05
roofmatl_Tar&Grv  3.649e+05   5.05e+04      7.219      0.000    2.66e+05    4.64e+05
roofmatl_WdShake  3.623e+05   5.27e+04      6.874      0.000    2.59e+05    4.66e+05
roofmatl_WdShngl  4.655e+05   5.19e+04      8.965      0.000    3.64e+05    5.67e+05
==============================================================================
Omnibus:                      307.148   Durbin-Watson:                   2.027
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2410.658
Skew:                           0.755   Prob(JB):                         0.00
Kurtosis:                       9.111   Cond. No.                     1.83e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.83e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

All features included are statistically signicant.

In [35]:
X4 = pd.concat([df3[['overallcond', 'grlivarea', 'garagecars']]], axis = 1)
y4 = df3['saleprice']


# Find the OLS
import statsmodels.api as sm
X4 = sm.add_constant(X4)

results = sm.OLS(y4, X4).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.626
Model:                            OLS   Adj. R-squared:                  0.625
Method:                 Least Squares   F-statistic:                     812.3
Date:                Tue, 26 May 2020   Prob (F-statistic):          2.78e-310
Time:                        21:48:54   Log-Likelihood:                -17826.
No. Observations:                1460   AIC:                         3.566e+04
Df Residuals:                    1456   BIC:                         3.568e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -3.012e+04   8174.403     -3.684      0.000   -4.62e+04   -1.41e+04
overallcond  2764.2363   1164.498      2.374      0.018     479.964    5048.508
grlivarea      79.1258      2.741     28.871      0.000      73.750      84.502
garagecars   4.285e+04   1955.033     21.916      0.000     3.9e+04    4.67e+04
==============================================================================
Omnibus:                      312.564   Durbin-Watson:                   2.036
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3924.016
Skew:                           0.621   Prob(JB):                         0.00
Kurtosis:                      10.935   Cond. No.                     1.04e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.04e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""